# Bem-Vindo ao Planeja SJC!

Aqui, você encontrará uma ampla gama de recursos para explorar e compreender a dinâmica da nossa cidade. Nossa plataforma oferece mapas interativos que permitem visualizar diferentes aspectos do espaço urbano, desde a infraestrutura até as áreas verdes, ajudando você a entender melhor o ambiente em que vive. Além disso, disponibilizamos dados demográficos detalhados, que fornecem uma visão abrangente sobre a população, suas características e necessidades. Esses dados são essenciais para que cidadãos, planejadores urbanos e gestores públicos possam tomar decisões informadas e embasadas.

### Sumário dos Gráficos

1 - População Total por Bairro e Zona

2 - Crescimento Populacional por Zona entre 2010 e 2022

3 - Pirâmide Etária

4 - Densidade Populacional por Região

### População Total por Bairro e Zona

### Crescimento Populacional por Zona entre 2010 e 2022

### Pirâmide Etária

### Densidade Populacional por Região